In [1]:
import cv2
import copy
import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd
from glob import glob
import os,sys,gc
import argparse
import json,time
import matplotlib.pyplot as plt
import torch
from torch import from_numpy
import subprocess
from pathlib import Path
from typing import NamedTuple
from math import ceil, floor
import ffmpeg

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


In [3]:
# openpose setup
from src import model
from src import util_add_fun as util
from src.body import Body
# from src.hand import Hand

In [4]:
# pretrained model of pytorch-openpose
openpose = Body('/home/bhennelly/Documents/QIN/thesis_project/pytorch-openpose-direct-v2/weights/body_pose_model.pth')

In [5]:
video_path = '/home/bhennelly/Documents/QIN/thesis_project/FLIR_fall'

files = os.listdir(video_path)
print(len(files))

44


In [15]:
# import random
# random.seed(2025)

# fall_videos = []
# normal_videos = []

# for file in files:
#     if 'FALL' in file:
#         fall_videos.append(file)
#     else:
#         normal_videos.append(file)  
# # print(len(fall_videos)) # 35
# # print(len(normal_videos)) # 9

# def get_random_sample(videos, ratio = 0.10):
#     n = max(1, int(len(videos)*ratio))
#     print(n)
#     return random.sample(videos, n)

# valid_fall_videos = get_random_sample(fall_videos, ratio = 0.10) # 3
# valid_normal_videos = get_random_sample(normal_videos, ratio = 0.10) # 1

# valid_videos = valid_fall_videos + valid_normal_videos
# print(valid_videos)

3
1
['FALL_31.avi', 'FALL_26.avi', 'FALL_29.avi', 'ADL_2.avi']


In [16]:
valid_videos = ['FALL_31.avi', 'FALL_26.avi', 'FALL_29.avi', 'ADL_2.avi']
train_videos = [x for x in files if x not in valid_videos]
print(len(train_videos))
print(len(valid_videos))

40
4


In [17]:
import os
import shutil

def clear_files(data_save_path, delete_folder=False):
    if os.path.exists(data_save_path):
        if delete_folder:
            shutil.rmtree(data_save_path)
        else:
            print(f"Directory {data_save_path} already exists.")

    if not os.path.exists(data_save_path):
        os.makedirs(data_save_path)
        os.makedirs(os.path.join(data_save_path, 'train', 'fall'), exist_ok=True)
        os.makedirs(os.path.join(data_save_path, 'train', 'normal'), exist_ok=True)
        os.makedirs(os.path.join(data_save_path, 'valid', 'fall'), exist_ok=True)
        os.makedirs(os.path.join(data_save_path, 'valid', 'normal'), exist_ok=True)

In [18]:
data_path = '/home/bhennelly/Documents/QIN/thesis_project/pytorch-openpose-direct-v2/datasets'

In [30]:
clear_files(data_path, delete_folder=True)

In [31]:
target_data = [[1,'FALL', 29],[2,'FALL', 28], [3,'FALL', 28], [4,'FALL', 43], [5,'FALL', 48], [6,'FALL', 29], [7,'FALL', 42],
               [8,'FALL', 29], [9,'FALL', 27], [10,'FALL', 26],  [11,'FALL', 21], [12,'FALL', 23], [13,'FALL', 22], [14,'FALL', 30],
               [15,'FALL', 25], [16,'FALL', 29], [17,'FALL', 26], [18,'FALL', 24], [19,'FALL', 25], [20,'FALL', 52], [21,'FALL', 30],
               [22,'FALL', 51], [23,'FALL', 32], [24,'FALL', 41], [25,'FALL', 43], [26,'FALL', 56], [27,'FALL', 37], [28,'FALL', 44],
               [29,'FALL', 51], [30,'FALL', 58],  [31,'FALL', 34], [32,'FALL', 47],  [33,'FALL', 43], [34,'FALL', 58],  [35,'FALL', 40],
               [1,'ADL',99999], [2,'ADL',99999], [3,'ADL',99999], [4,'ADL',99999],  [5,'ADL',99999], [6,'ADL',99999], [7,'ADL',99999],
               [8,'ADL',99999], [9,'ADL',99999]]   

target_data = pd.DataFrame(target_data, columns=['videoID','fileTarget','fall_time'])
target_data['videoID'] = target_data['videoID'].apply(str)
target_data['fall_time'] = target_data['fall_time'].apply(lambda x:round(x,2))
target_data['new_videoID'] = target_data[['fileTarget','videoID']].apply(lambda row: row[0] + '_' + row[1] + '.avi', axis=1)
target_data = target_data[['new_videoID','fall_time']].drop_duplicates().reset_index(drop=True)
print(target_data.shape)
print(target_data.head())

(44, 2)
  new_videoID  fall_time
0  FALL_1.avi         29
1  FALL_2.avi         28
2  FALL_3.avi         28
3  FALL_4.avi         43
4  FALL_5.avi         48


/tmp/ipykernel_2927738/551456177.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target_data['new_videoID'] = target_data[['fileTarget','videoID']].apply(lambda row: row[0] + '_' + row[1] + '.avi', axis=1)


In [32]:
stride = 8 # 步长8，在做补全的时候，一般都是8的倍数？
upsample_ratio = 4 # 这是什么意思？
num_keypoints = 18
cpu=False
height_size = 256

In [33]:
import cv2
import numpy as np
import math

def draw_bodypose(canvas, candidate, subset):
    # 初始化一个全黑的画布（原图尺寸）
    black_canvas = np.zeros_like(canvas)
    
    stickwidth = 4
    limbSeq = [[2, 3], [2, 6], [3, 4], [4, 5], [6, 7], [7, 8], [2, 9], [9, 10], \
               [10, 11], [2, 12], [12, 13], [13, 14], [2, 1], [1, 15], [15, 17], \
               [1, 16], [16, 18], [3, 17], [6, 18]]
    
    # 1. 只画骨架（不画关键点圆圈）
    for i in range(17):  # 遍历所有骨架连接
        for n in range(len(subset)):
            index = subset[n][np.array(limbSeq[i]) - 1]
            if -1 in index:
                continue
            
            # 获取两个关键点坐标
            Y = candidate[index.astype(int), 0]  # x坐标
            X = candidate[index.astype(int), 1]  # y坐标
            mX = np.mean(X)
            mY = np.mean(Y)
            length = ((X[0] - X[1]) ** 2 + (Y[0] - Y[1]) ** 2) ** 0.5
            angle = math.degrees(math.atan2(X[0] - X[1], Y[0] - Y[1]))
            
            # 用白色绘制骨架
            polygon = cv2.ellipse2Poly((int(mY), int(mX)), (int(length / 2), stickwidth), int(angle), 0, 360, 1)
            cv2.fillConvexPoly(black_canvas, polygon, (255, 255, 255))  # 白色骨架
    
    # 2. 调整分辨率为 (128, 128)，虽然存储的是彩色图，但实际在提取的时候是灰度图
    resized_canvas = cv2.resize(black_canvas, (128, 128), interpolation=cv2.INTER_NEAREST)
    
    # # 3. 转换成灰度图
    # if resized_canvas.ndim == 3:  # 如果是彩色图 (H, W, 3)
    #     resized_canvas = cv2.cvtColor(resized_canvas, cv2.COLOR_BGR2GRAY) # 转换成灰度图

    return resized_canvas

In [34]:
def get_train_datasets(train_videos, data_tag = '', target_data = pd.DataFrame(), data_path = ''):

    for video in train_videos:
        videoID = video.split('.')[0].replace('_','')

        video_addr = os.path.join(video_path, video)

        cap = cv2.VideoCapture(video_addr)
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        frame_interval = 2
        frame_count = 0

        while cap.isOpened() and frame_count < total_frames:

            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
            ret, img = cap.read()

            if not ret:
                break

            timestamps = frame_count/fps
            fall_time = target_data[target_data['new_videoID']==video]['fall_time'].values[0]
            if timestamps >= fall_time:
                label_tag = 'fall'
            else:
                label_tag = 'normal'
            
            canvas = img.copy()

            candidate, subset = openpose(img)

            if len(candidate)>=3:
                skeleton_img = draw_bodypose(canvas, candidate, subset)
                tm = time.time()
                tm = int(round(tm*1000))
                t = videoID+'_'+str(tm)
                img_save_name = f'{data_tag}/{label_tag}/{t}.jpg'
                cv2.imwrite(os.path.join(data_path, img_save_name), skeleton_img)
                
            frame_count+=frame_interval
        

In [35]:
get_train_datasets(train_videos, data_tag = 'train', target_data = target_data, data_path = data_path)

In [36]:
get_train_datasets(valid_videos, data_tag = 'valid', target_data = target_data, data_path = data_path)

In [37]:
import gc
gc.collect()

785

In [38]:
train_fall = len(os.listdir(os.path.join(data_path, 'train', 'fall')))
print(train_fall)
train_normal = len(os.listdir(os.path.join(data_path, 'train', 'normal')))
print(train_normal)
valid_fall = len(os.listdir(os.path.join(data_path, 'valid', 'fall')))
print(valid_fall)
valid_normal = len(os.listdir(os.path.join(data_path, 'valid', 'normal')))
print(valid_normal)

161
3731
11
527


In [39]:
print(train_fall/train_normal)
print(valid_fall/valid_normal)

# 0.03776360961255518
# 0.06077348066298342

# 0.043151969981238276
# 0.020872865275142316

0.043151969981238276
0.020872865275142316
